In [1]:
from transformers import DataCollatorWithPadding

In [2]:
from transformers import AutoTokenizer

In [3]:
from datasets import load_dataset

In [4]:
train_dataset = load_dataset("csv", data_files="books_train.csv")["train"]
test_dataset = load_dataset("csv", data_files="books_test.csv")["train"]

In [5]:
train_dataset

Dataset({
    features: ['body', 'label'],
    num_rows: 200
})

In [6]:
test_dataset

Dataset({
    features: ['body', 'label'],
    num_rows: 100
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def preprocess_function(examples):
   return tokenizer(examples["body"], truncation=True)
 
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Training the model 

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import numpy as np
import evaluate
 

In [12]:
def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)
   f1 = load_f1.compute(predictions=predictions, references=labels)
   return {"accuracy": accuracy, "f1": f1}

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
from transformers import TrainingArguments, Trainer
 
repo_name = "finetuning-sentiment-model-3000-samples"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [15]:
trainer.train()

TrainOutput(global_step=26, training_loss=0.6157895601712741, metrics={'train_runtime': 22.2671, 'train_samples_per_second': 17.964, 'train_steps_per_second': 1.168, 'total_flos': 51536027486496.0, 'train_loss': 0.6157895601712741, 'epoch': 2.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.5823735594749451,
 'eval_accuracy': {'accuracy': 0.72},
 'eval_f1': {'f1': 0.8372093023255814},
 'eval_runtime': 3.3462,
 'eval_samples_per_second': 29.884,
 'eval_steps_per_second': 2.092,
 'epoch': 2.0}

In [16]:
trainer.evaluate()

{'eval_loss': 0.584126353263855,
 'eval_accuracy': {'accuracy': 0.72},
 'eval_f1': {'f1': 0.8372093023255814},
 'eval_runtime': 3.1473,
 'eval_samples_per_second': 31.774,
 'eval_steps_per_second': 2.224,
 'epoch': 2.0}